In [1]:
%reload_ext autoreload
%autoreload 2

In [40]:
from BaconShor3CXheavyhex import BS3CXheavyhex
from utils import build_stim_circuit

def onlyBS(d, T,logical_prep, offset = -2+6j,
            num_qubits = -1, 
            virt_link = False,
            CXerror= 0, Rerror= 0, singleQerror=0):
    """We still do measurements on the other sublattice, but there are no additional CX layers from the 3CX code"""
    
    code = BS3CXheavyhex(d=d, T=T, logical_prep=['Z',logical_prep], logical_meas=['Z',logical_prep], offset=offset,
                            num_qubits=num_qubits, 
                            virt_link=virt_link,
                            CX_schedule = ['BSZ0', 'BSZ1','BSX1', 'BSX0'],
                            CXerror=CXerror, Rerror=Rerror,singleQerror=singleQerror, idleerror=1e-2)
    return code

In [41]:
code = onlyBS(d=3, T=2, logical_prep = 'Z' , CXerror=0, Rerror=0, singleQerror=0)

stim_circuit,relevant_nodes = build_stim_circuit(code,return_relevant_nodes=True)
stim_DEM = stim_circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=False)

In [42]:
stim_circuit

stim.Circuit('''
    H 3 5 10 11 28 29 30 33 34 35
    CX 3 17 28 3 3 17 28 3 4 18 29 4 4 18 29 4 5 19 30 5 5 19 30 5 9 20 33 9 9 20 33 9 10 21 34 10 10 21 34 10 11 22 35 11 11 22 35 11 6 20 28 6 6 20 28 6 7 21 29 7 7 21 29 7 8 22 30 8 8 22 30 8 12 23 33 12 12 23 33 12 13 24 34 13 13 24 34 13 14 25 35 14 14 25 35 14 3 26 17 3 3 26 17 3 4 27 18 4 4 27 18 4 9 31 20 9 9 31 20 9 10 32 21 10 10 32 21 10 15 36 23 15 15 36 23 15 16 37 24 16 16 37 24 16 1 26 18 1 1 26 18 1 2 27 19 2 2 27 19 2 7 31 21 7 7 31 21 7 8 32 22 8 8 32 22 8 13 36 24 13 13 36 24 13 14 37 25 14 14 37 25 14
    TICK
    H 4 5 9 11
    M 4 5 9 11 3 10 15 16
    PAULI_CHANNEL_1(0.0025, 0.0025, 0.0025) 0 1 2 6 7 8 12 13 14 17 18 19 20 21 22 23 24 25
    R 4 5 9 11 3 10 15 16
    H 4 5 9 11 28 29 30 33 34 35
    M 28 29 30 33 34 35 26 27 31 32 36 37
    R 28 29 30 33 34 35 26 27 31 32 36 37
    H 28 29 30 33 34 35
    TICK
    CX 18 1 1 26 18 1 1 26 19 2 2 27 19 2 2 27 21 7 7 31 21 7 7 31 22 8 8 32 22 8 8 32 24 13 13 36 24 13

In [37]:
stim_DEM.diagram(type='matchgraph-3d')

{"accessors":[{"bufferView":0,"byteOffset":0,"componentType":5126,"count":17,"max":[0,0.400000005960464,0.400000005960464],"min":[0,-0.400000005960464,-0.400000005960464],"name":"circle_loop","type":"VEC3"},{"bufferView":1,"byteOffset":0,"componentType":5126,"count":324,"max":[73,37,10.4721355438232],"min":[41,15,-4.4721360206604],"name":"buf_scattered_lines","type":"VEC3"},{"bufferView":2,"byteOffset":0,"componentType":5126,"count":72,"max":[57,15,10.4721355438232],"min":[57,5,-4.4721360206604],"name":"buf_red_scattered_lines","type":"VEC3"}],"asset":{"version":"2.0"},"bufferViews":[{"buffer":0,"byteLength":204,"byteOffset":0,"name":"circle_loop","target":34962},{"buffer":1,"byteLength":3888,"byteOffset":0,"name":"buf_scattered_lines","target":34962},{"buffer":2,"byteLength":864,"byteOffset":0,"name":"buf_red_scattered_lines","target":34962}],"buffers":[{"byteLength":204,"name":"circle_loop","uri":"data:application/octet-stream;base64,AAAAAM3MzD4AAAAAAAAAAOU1vT5Fvxw+AAAAAMPQkD7D0JA+AAAAAES/HD7lNb0+AAAAAPIwlrLNzMw+AAAAAEe/HL7lNb0+AAAAAMPQkL7D0JA+AAAAAOc1vb5Avxw+AAAAAM3MzL7yMBazAAAAAOU1vb5Evxy+AAAAAMHQkL7E0JC+AAAAADy/HL7nNb2+AAAAAPLkozHNzMy+AAAAAEm/HD7kNb2+AAAAAMbQkD6/0JC+AAAAAOY1vT5Evxy+AAAAAM3MzD4AAAAA"},{"byteLength":3888,"name":"buf_scattered_lines","uri":"data:application/octet-stream;base64,AABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAAcEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAAAAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAEBAAABkQgAA2EEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAEBAAABkQgAAcEEAAMBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAAAAAABkQgAA2EEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAcEEAAMBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAA2EEAAMBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAAcEEAAMBAAABkQgAA2EEAAMBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAMBAAABkQu/GD0LejSdBAABMQgAAqEEAAEBAAAB8QgAAqEEAAEBAAABkQgAA2EEAAMBAAABkQu/GD0LejSdBAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAA2EEAAMBAAABkQu/GD0LejSdBAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAA2EEAAEBAAABkQgAAFEIAAEBAAABkQgAA2EEAAAAAAABkQu/GD0K9G4/AAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAA2EEAAMBAAABkQu/GD0LejSdBAABkQgAAcEEAAAAAAABkQgAAcEEAAEBAAABkQgAAcEEAAAAAAABkQgAAcEEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAAcEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAAAAAABkQgAAcEEAAAAAAABkQgAA2EEAAAAAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAAAAAABMQgAAqEEAAEBAAAAkQgAAqEEAAEBAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAAAAAAB8QgAAqEEAAEBAAACSQgAAqEEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAEBAAABkQgAAcEEAAAAAAABkQgAA2EEAAEBAAABMQgAAqEEAAEBAAAAkQgAAqEE